# Ejercicio: índice de marginación

Buscaremos representaciones de dimensión uno para los datos de marginación (2010) a nivel municipio de CONAPO.

In [ ]:
%autosave 0
import pandas as pd
import numpy as np
from plotnine import *

## 1. Descripción de datos

Leemos los datos y vemos una descripción

In [ ]:
marginacion = pd.read_csv("../datos/imm-2010.csv")
descrip = pd.read_csv("../datos/imm-2010-descrip.csv", skiprows=2)
pd.set_option('max_colwidth', 200)
descrip

In [ ]:
marginacion

Calculamos algunos resúmenes de las variables numéricas (excluyendo el índice de marginación oficial)

In [ ]:
marg_tabla = marginacion.loc[:,'ANALF':'PO2SM']
marg_tabla
# Cuantiles 0.1, 0.5 (mediana) y 0.9 , 0.99
marg_tabla.quantile([0, .10, 0.5, 0.9, 0.99, 1]).stack().unstack(0).round(1)

Todas las variables son porcentajes, y sus escalas no son tan distintas (argumenta).

## 2. Aproximación de rango 1

Nos interesa ahora construir una aproximación de rango 1 a esta tabla de datos de municipios. La idea principal
es que si construimos una aproximación $X\approx uv^t$, el vector $u$, cuya longitud es el número de municipios,
nos de una **medición general de marginación**. Tendremos que checar que esta interpretación es la correcta.

En primer lugar, no daremos tratamiento a los indicadores. Usamos la función svd para extraer la primera componente (ver notas)

In [ ]:
from numpy.linalg import svd
u, s, v_t = ## aquí tu código
signo = -1 ## por qué cambiar el signo?
u = signo * u[:, 0].reshape(-1, 1)
v = signo * v_t[0, :].reshape(-1, 1)
s = s[0]

Examina los pesos en el vector $v$ que corresponde a las variables:

In [ ]:
pesos_variables = pd.DataFrame(v, index = marg_tabla.columns).rename(columns={0:'v'})
pesos_variables.sort_values(by='v')

**Pregunta**: ¿cómo interpretas estos números en término de la tabla de datos?

## 3. Calidad de la aproximación

Calcula la aproximación de rango 1 y compara contra los observados en la tabla

In [ ]:
X_1 = # aqui tu código de la aproximación rango 1

In [ ]:
X_1.shape

In [ ]:
# Calcula residuales
R = # escribe tu código
R.shape

In [ ]:
# Calcula porcentaje de tamaño de residuales vs tamaño de la tabla
error_relativo = np.sum(R**2) / np.sum(marg_tabla.values**2)
error_relativo.round(2)

## 4. Examinar scores de municipios

Haz un histograma o gráfica de cuantiles de los scores de los municipios

In [ ]:
# Agrega a la tabla los scores
marginacion['score_mun'] = u
# Grafica
ggplot(# completa) + # completa

**Pregunta**: considerando el vector $v$ de pesos que vimos arriba y estos scores, ¿cómo interpretas el score de 
    municipios que acabamos de calcular?

## 5. Municipios de score bajos y altos

Encuentra el 1% municipios con score más bajo y el 1% con score más alto

In [ ]:
marginacion['rank_score'] = # aquí tu código, puedes hacerlo de otra manera
marginacion.loc[ marginacion['rank_score'] <= 0.01 ]

In [ ]:
marginacion.loc[ marginacion['rank_score'] >= 0.99 ]

**Pregunta**: ¿donde están ubicados los diez menos marginados y los diez más marginados? ¿Cómo se ven sus variables indicadoras en cada caso?

## 6. Normalización de variables

Como vimos antes, las variables que usamos tienen distintos niveles y dispersión. Esto produce que pesen más en el ajuste variables que tienen valores altos (pues influyen más en el error de aproximación). Podemos fijar la escala de las variables de entrada para evitar este efecto.

Hay varias maneras de hacer esto (por ejemplo, restando media y dividiendo por desviación estándar, que veremos más adelante). Una manera es normalizando para el rango de las variables esté entre 0 y 1, excluyendo valores atípicos

In [ ]:
#normaliza tabla para que esté aproximadamente entre 0 y 1
#usamos un cuantil en lugar del máximo para no aplastar tanto 
#debido a atípicos superiores
max_c = marg_tabla.quantile(0.95)
max_c

In [ ]:
marg_tabla_norm = marg_tabla / max_c
marg_tabla_norm

Ahora recalcula la aproximación de rango 1 y compara el índice que encontraste con el IMC reportado en la tabla (el valor oficial)

In [ ]:
u, s, v_t = # aqui tu código
signo = -1
u = # completa 
v = # completa
s = #completa 
u

In [ ]:
# Agrega a la tabla los scores
marginacion['score_mun_norm'] = u
# Grafica comparando con el IM
ggplot(#completa)

Para comparar qué tan similares son las ordenaciones sugeridas por nuestro indice y el oficial, podemo usar la tau de Kendall, que mide que fracción de pares posible están en el mismo orden en los dos índices: 

In [ ]:
from scipy.stats import kendalltau
kendalltau(marginacion['score_mun_norm'], marginacion['IM'])

Finalmente, los pesos nuevos son:

In [ ]:
pesos_variables = pd.DataFrame(v, index = marg_tabla.columns).rename(columns={0:'v'})
pesos_variables.sort_values(by='v')

## 7. Pesos y scores

Veremos ahora un aspecto importante de los vectores $u$ de scores y $v$ de pesos, y de por qué su nombre.
Empezamos recordando los pesos del vector $v$:

In [ ]:
pesos_variables = pd.DataFrame(v, index = marg_tabla.columns).rename(columns={0:'v'})
pesos_variables.sort_values(by='v')

Ahora creamos un índice ponderando las variables originales por estos pesos. Esto se hace multiplicando la tabla original $X$ por los pesos $v$:

In [ ]:
X = marg_tabla_norm.values
indice_ponderado = #multiplica X por v
indice_ponderado.shape

Y ahora comparamos con los scores $v$ que calculamos antes:

In [ ]:
comp_df = # haz un dataframe con el indice ponderado
comp_df['sigma*u'] = # agrega u escalado por s
comp_df

### Los scores $u$ son las variables originales ponderadas por los pesos $v$ (módulo una constante) 

Discutiremos más adelante por qué es cierta en general esta afirmación que nos permite interpetar de manera simple scores y pesos.

## 8. Análisis estandarizando variables

**Pregunta**: repite ahora estandarizando con media y desviación estándar

In [ ]:
# aquí tu código, encuentra u, v y sigma
# estandariza la tabla
marg_tabla_est = # estandariza la tabla
# svd
#
# Agrega a la tabla los scores, estandarizando u
#
# Grafica comparando con el IM
#

## 9. Segunda dimensión latente

El índice de marginación nos da la primera aproximación a la tabla de datos de marginación por municipio, y es la mejor
aproximación de rango 1. Buscaremos ahora mejorar nuestra solución agregando otra componente de rango 2

In [ ]:
from numpy.linalg import svd
# puedes usar la tabla de standarizados también si la calculaste
# en el ejercicio anterior
U, S, V_t = svd(marg_tabla_norm.values)
signo = -1
u_1 = signo * U[:, 0].reshape(-1, 1)
v_1 = signo * V_t[0, :].reshape(-1, 1)
s_1 = S[0]
v_1

La calidad porcentual de aproximación, por lo que acabamos de notar, se calcula como

In [ ]:
X_norma = # calcula la norma de X
X_norma

In [ ]:
calidad = # calcula calidad de rango 1
calidad

Ahora consideramos la segunda dimensión latente

In [ ]:
u_2 = # extrae dimensión 2
v_2 = #
s_2 = #

pesos_variables_2 = pd.DataFrame(v_2, index = marg_tabla_norm.columns).rename(columns={0:'v_2'})
pesos_variables_2.sort_values(by='v_2')

**Pregunta**: interpeta estos pesos. ¿Cuándo el score de un municipio es alto en esta dimensión? ¿Cuál es el patrón en los datos que está capturando esta segunda dimensión? 

Calcula ahora la calidad de la representación con dos dimensiones latentes

In [ ]:
calidad = # calcula calidad con dos dimensiones
calidad

**Pregunta**. qué tanto aporta esta segunda dimensión en comparación a la primera?

Ahora checa que los dos vectores $u_1$ y $u_2$ son ortogonales (igual para los $v$)

In [ ]:
np.matmul(u_1.transpose(), u_2)
np.matmul(v_1.transpose(), v_2)

## 10. Interpretación de segunda dimensión latente

Usa el siguiente código para interpetar la segunda dimensión que encontramos

In [ ]:
marginacion = marginacion.drop(['score_mun', 'rank_score', 'score_mun_norm', 'score_mun_est'], axis = 1)

In [ ]:
marginacion['score_mun_1'] = u_1
marginacion['score_mun_2'] = u_2
marginacion['rank_score_2'] = marginacion['score_mun_2'].rank(method = 'first') / len(marginacion)
marginacion.loc[ marginacion['rank_score_2'] >= 0.997 ]

In [ ]:
marginacion.loc[ marginacion['rank_score_2'] <= 0.003]

In [ ]:
marginacion.loc[marginacion['NOM_MUN'].isin(['San Andres Duraznal', 'San Lorenzo Cuaunecuiltitla', 'Chinipas', 'Batopilas'])]

In [ ]:
(ggplot(marginacion, aes('score_mun_1', 'score_mun_2', color='OVSEE')) + geom_point())